In [393]:
import pandas as pd

data = pd.read_excel('data_set.xlsx')

In [394]:
def replace_binary_value(b):
    if b and pd.notnull(b):
        return 1
    else:
        return -1

data.iloc[:, 12:16] = data.iloc[:, 12:16].agg(lambda row: row.agg(replace_binary_value, axis=0), axis=1)
data_kal_obl = data[data['REGION'] == 'Калужская область']
data_tul_obl = data[data['REGION'] == 'Тульская область']

In [395]:
import seaborn as sns

#sns.pairplot(data_kal_obl.iloc[:, 3:17])

In [396]:
data_kaluga = data_kal_obl[data_kal_obl['CITY'] == 'КАЛУГА']
data_tula = data_tul_obl[data_tul_obl['CITY'] == 'ТУЛА']
data_kal_obl = data_kal_obl[data_kal_obl['CITY'] != 'КАЛУГА']
data_tul_obl = data_tul_obl[data_tul_obl['CITY'] != 'ТУЛА']

In [397]:
X = data_kaluga[data_kaluga['cashbox_daily'].notnull()].iloc[:, 6:18]

In [398]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57 entries, 0 to 239
Data columns (total 11 columns):
cashbox_MAX                57 non-null float64
cashbox_trend              55 non-null float64
hh_500                     57 non-null float64
value1                     57 non-null int64
value2                     57 non-null int64
POPULATION                 57 non-null int64
NEAR_Malls                 57 non-null int64
NEAR_Business_centers      57 non-null int64
NEAR_metro_rjd             57 non-null int64
NEAR_Stations              57 non-null int64
macro_salary_avg_yearly    57 non-null int64
dtypes: float64(3), int64(8)
memory usage: 5.3 KB


In [399]:
del X['POPULATION']
del X['macro_salary_avg_yearly']

In [400]:
import numpy as np

X.fillna(0, inplace=True)
X = (X - X.mean()) / X.std()

In [401]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse

lasso = Lasso(0.05)
ridge = Ridge()
lin_reg = LinearRegression()

In [402]:
y = data_kaluga[data_kaluga['cashbox_daily'].notnull()].cashbox_daily
y = (y - y.mean()) / y.std()

In [403]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

lasso.fit(X_train, y_train)
print(mse(y_test, lasso.predict(X_test)), mae(y_test, lasso.predict(X_test)))

0.14214809696999325 0.3097845515954967


In [404]:
lasso.fit(X, y)
lasso.coef_

array([ 0.7950587 ,  0.        ,  0.        ,  0.04516096,  0.05593081,
       -0.        ,  0.        ,  0.01609916, -0.06898949])

In [405]:
lasso.fit(X.drop(labels='value1', axis=1), y)
lasso.coef_

array([ 0.7949395 ,  0.        ,  0.        ,  0.09664327, -0.        ,
        0.        ,  0.01323896, -0.07149039])

In [406]:
lasso.fit(X.drop(labels='value2', axis=1), y)
lasso.coef_

array([ 0.79889328,  0.        ,  0.00198145,  0.09487306, -0.        ,
        0.        ,  0.01836106, -0.0664145 ])

In [407]:
X.drop(labels=['cashbox_trend', 'value2', 'NEAR_Malls', 'NEAR_Business_centers'], axis=1, inplace=True)

In [408]:
X.head()

,cashbox_MAX,hh_500,value1,NEAR_metro_rjd,NEAR_Stations
0,1.494381,1.675132,1.290828,-0.132453,1.044805
2,1.212329,0.744297,1.307995,-0.132453,-0.940325
3,1.212329,1.443183,1.290828,-0.132453,-0.940325
4,0.366173,1.055299,1.239327,-0.132453,1.044805
6,-0.479983,1.678173,1.101992,-0.132453,1.044805


In [409]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse
from sklearn.tree import ExtraTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor


def test_regressor(X, y, regressor):
    mean_mse = 0
    mean_mae = 0
    for i in range(100):
        X_train , X_test, y_train, y_test = train_test_split(X, y)
        regressor.fit(X_train, y_train)
        mean_mse += mse(y_test, regressor.predict(X_test))
        mean_mae += mae(y_test, regressor.predict(X_test))
    print('MSE = ', mean_mse/100)
    print('MAE = ', mean_mae/100)


def getBenchmark(X, y, lasso_coef=0.05):
    lasso = Lasso(lasso_coef)
    ridge = Ridge()
    lin_reg = LinearRegression()
    des_tree = DecisionTreeRegressor()
    extra_tree = ExtraTreeRegressor()
    knn = KNeighborsRegressor()
    
    
    print('LinReg:')
    test_regressor(X, y, lin_reg)
    print('Lasso:')
    test_regressor(X, y, lasso)
    print('Ridge:')
    test_regressor(X, y, ridge)
    print('ExtraTree:')
    test_regressor(X, y, extra_tree)
    print('KNN:')
    test_regressor(X, y, knn)
    print('DesTree:')
    test_regressor(X, y, des_tree)
    

In [456]:
def getPredict(data, regressor, labels_to_delete, target_name):
    X = data[data[target_name].notnull()].iloc[:, 5:18]
    X.drop(labels=labels_to_delete, axis=1, inplace=True)
    y = data[data[target_name].notnull()][target_name]

    regressor.fit(X, y)
    
    df = data[data[target_name].isnull()].iloc[:, 5:18]
    df['cashbox_MAX'] = 0
    df.drop(labels=labels_to_delete, axis=1, inplace=True)
    
    arr = regressor.predict(df)
    
    forResult = data[data[target_name].isnull()]['ADDRESS']
    indexes = [x for x in range(len(arr))]
    forResult.index = indexes
    result = {}
    for i in range(len(arr)):
        result[forResult.loc[i]] = arr[i]

    return result

In [457]:
l = ['cashbox_trend', 'value2', 'NEAR_Malls', 'NEAR_Business_centers', 'POPULATION', 'macro_salary_avg_yearly',\
     'Postamat_trend']
target = 'cashbox_daily'
getPredict(data_kaluga, Ridge(), l, target)

{'г.Калуга, Гагарина ул., 1': 0.04060296160984578,
 'г.Калуга, Мичурина ул., 11': 0.19420493040719222,
 'г.Калуга, Пушкина ул., 3 п.68': 0.057365657337339115,
 'г.Калуга, Дзержинского ул., 38': 0.16822704922893195,
 'г.Калуга, Гурьянова ул., 7': 0.4407016289184149,
 'г.Калуга, Московская ул., 336А': 0.012077628402074936,
 'г.Калуга, д.Шопино': 0.06830273896864725,
 'г.Калуга, 40-летия Октября ул, 3': -0.2526299947932209,
 'г.Калуга, Салтыкова-Щедрина ул, 74': -0.30555644308325824}